In [69]:
!pip install seaborn

In [70]:
%pip install scikit-learn pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [72]:
df = pd.read_csv('aie.csv')

In [73]:
df.head()

,Job_Title,Average_Salary,Years_Experience,Education_Level,AI_Exposure_Index,Tech_Growth_Factor,Automation_Probability_2030,Risk_Category,Skill_1,Skill_2,Skill_3,Skill_4,Skill_5,Skill_6,Skill_7,Skill_8,Skill_9,Skill_10
0,Security Guard,45795,28,Master's,0.18,1.28,0.85,High,0.45,0.10,0.46,0.33,0.14,0.65,0.06,0.72,0.94,0.00
1,Research Scientist,133355,20,PhD,0.62,1.11,0.05,Low,0.02,0.52,0.40,0.05,0.97,0.23,0.09,0.62,0.38,0.98
2,Construction Worker,146216,2,High School,0.86,1.18,0.81,High,0.01,0.94,0.56,0.39,0.02,0.23,0.24,0.68,0.61,0.83
3,Software Engineer,136530,13,PhD,0.39,0.68,0.60,Medium,0.43,0.21,0.57,0.03,0.84,0.45,0.40,0.93,0.73,0.33
4,Financial Analyst,70397,22,High School,0.52,1.46,0.64,Medium,0.75,0.54,0.59,0.97,0.61,0.28,0.30,0.17,0.02,0.42


In [74]:
final_df = df.drop(columns=['Skill_1', 'Skill_2', 'Skill_3', 'Skill_4', 'Skill_5', 'Skill_6', 'Skill_7', 'Skill_8', 'Skill_9', 'Skill_10'])

In [75]:
final_df.head()

,Job_Title,Average_Salary,Years_Experience,Education_Level,AI_Exposure_Index,Tech_Growth_Factor,Automation_Probability_2030,Risk_Category
0,Security Guard,45795,28,Master's,0.18,1.28,0.85,High
1,Research Scientist,133355,20,PhD,0.62,1.11,0.05,Low
2,Construction Worker,146216,2,High School,0.86,1.18,0.81,High
3,Software Engineer,136530,13,PhD,0.39,0.68,0.60,Medium
4,Financial Analyst,70397,22,High School,0.52,1.46,0.64,Medium


In [76]:
X = final_df.drop('Risk_Category', axis=1)
y = final_df['Risk_Category']

In [77]:
final_df["Education_Level"].unique()

array(["Master's", 'PhD', 'High School', "Bachelor's"], dtype=object)

In [78]:
X.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Job_Title                    3000 non-null   object 
 1   Average_Salary               3000 non-null   int64  
 2   Years_Experience             3000 non-null   int64  
 3   Education_Level              3000 non-null   object 
 4   AI_Exposure_Index            3000 non-null   float64
 5   Tech_Growth_Factor           3000 non-null   float64
 6   Automation_Probability_2030  3000 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 164.2+ KB
<class 'pandas.core.series.Series'>
RangeIndex: 3000 entries, 0 to 2999
Series name: Risk_Category
Non-Null Count  Dtype 
--------------  ----- 
3000 non-null   object
dtypes: object(1)
memory usage: 23.6+ KB


In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [90]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2400 entries, 1267 to 274
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Job_Title                    2400 non-null   object 
 1   Average_Salary               2400 non-null   int64  
 2   Years_Experience             2400 non-null   int64  
 3   Education_Level              2400 non-null   object 
 4   AI_Exposure_Index            2400 non-null   float64
 5   Tech_Growth_Factor           2400 non-null   float64
 6   Automation_Probability_2030  2400 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 150.0+ KB


In [ ]:
ct = ColumnTransformer(
    transformers=[
        ("edu_enc",
         OrdinalEncoder(categories=[["High School", "Bachelor's", "Master's", "PhD"]]),
         ["Education_Level"]),

        ("job_ohe",
         OneHotEncoder(handle_unknown="ignore"),
         ["Job_Title"]),

        ("scaler",
         StandardScaler(),
         ["Average_Salary", "Years_Experience"])
    ],
    remainder="passthrough"
)

In [82]:
from sklearn.linear_model import LogisticRegression, SGDRegressor

In [83]:
pipe = Pipeline(steps=[
    ("step1", ct),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [84]:
pipe2 = Pipeline(steps=[
    ("step1", ct),
    ("classifier", LogisticRegression())
])

In [85]:
pipe3 = Pipeline(steps=[
    ("step1", ct),
    ("classifier", SGDRegressor())
])

In [89]:
pipe.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe

In [ ]:
pipe2.fit(X_train, y_train)

c:\Users\Nani\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,steps,"[('step1', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('edu_enc', ...), ('job_ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
y2_pred = pipe2.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y2_pred))
print("recall:", recall_score(y_test, y2_pred, average='weighted'))
print("precision_score", precision_score(y_test, y2_pred, average='weighted'))

Accuracy: 0.9833333333333333
recall: 0.9833333333333333
precision_score 0.9835741314069488


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0


In [ ]:
X_test
y_test

2782       Low
2795    Medium
1774    Medium
2870      High
424     Medium
         ...  
2695    Medium
2104    Medium
2494    Medium
427        Low
1026    Medium
Name: Risk_Category, Length: 600, dtype: object

In [ ]:
mapping = {"Low": 0, "Medium": 1, "High": 2}
y_train_enc = y_train.map(mapping)
y_test_enc  = y_test.map(mapping)


In [ ]:
pipe3.fit(X_train,y_train)

ValueError: could not convert string to float: 'Medium'